In [19]:
import fitz
from collections import Counter

In [20]:
pdf_address = "li.pdf"
#pdf_address = "judea.pdf"
pdf_doc = fitz.open(pdf_address)

In [21]:
page = pdf_doc.load_page(10)
page_text = page.get_text("text")
page_text

'1 Introduction\n3\nelectrophysiology to compensate the low temporal resolution. Figure 2 is an exam-\nple. Intra-cellular recording directly measures neuronal activities, usually taken as\nground truth. Calcium imaging records fluorescence signals as an indicator of neu-\nronal spiking activity, which is indirect. Fluorescence signals and neuronal spikes\ndiffer from each other. Figure 3 shows single action potential and its evoked calcium\nsignal, obtained from recording rats’ calyx of Held - a large synaptic terminal in\nbrainstem slice. We see action potential is quick (in ms) with a short decay, while\nfluorescence signal (in this case, Fura-2) is slow in several seconds with a longer\ndecay. The fluorescence signal is much noisier than the action potential. The cal-\ncium indicator Section 1.1.2 is one key point to facilitate functional imaging with\nalive animals. For example, with a cranial window over visual cortex of a transgenic\nmouse, it is able to image large scale of neu

In [22]:
page_blocks = page.get_text("blocks")
page_blocks


[(113.38600158691406,
  46.6934814453125,
  510.26995849609375,
  62.151676177978516,
  '1 Introduction\n3\n',
  0,
  0),
 (113.00399780273438,
  82.85345458984375,
  512.27734375,
  293.4193115234375,
  'electrophysiology to compensate the low temporal resolution. Figure 2 is an exam-\nple. Intra-cellular recording directly measures neuronal activities, usually taken as\nground truth. Calcium imaging records fluorescence signals as an indicator of neu-\nronal spiking activity, which is indirect. Fluorescence signals and neuronal spikes\ndiffer from each other. Figure 3 shows single action potential and its evoked calcium\nsignal, obtained from recording rats’ calyx of Held - a large synaptic terminal in\nbrainstem slice. We see action potential is quick (in ms) with a short decay, while\nfluorescence signal (in this case, Fura-2) is slow in several seconds with a longer\ndecay. The fluorescence signal is much noisier than the action potential. The cal-\ncium indicator Section 1.1.2 is

In [176]:
page_dicts = page.get_text("dict")

In [222]:
len(page_dicts["blocks"])

7

In [24]:
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

In [25]:
blocks = page.get_text("dict", flags=11)["blocks"]
page_font_sizes = []
for b in blocks:  # iterate through the text blocks
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            print("")
            page_font_sizes.append(s["size"])
            font_properties = "Font: '%s' (%s), size %g, color #%06x" % (
                s["font"],  # font name
                flags_decomposer(s["flags"]),  # readable font flags
                s["size"],  # font size
                s["color"],  # font color
            )
            print("Text: '%s'" % s["text"])  # simple print of text
            print(font_properties)


Text: '1Introduction'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: '3'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'electrophysiologytocompensatethelowtemporalresolution.Figure2isanexam-'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'ple.Intra-cellularrecordingdirectlymeasuresneuronalactivities,usuallytakenas'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'groundtruth.Calciumimagingrecordsfluorescencesignalsasanindicatorofneu-'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'ronalspikingactivity,whichisindirect.Fluorescencesignalsandneuronalspikes'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000

Text: 'differfromeachother.Figure3showssingleactionpotentialanditsevokedcalcium'
Font: 'LMRoman10-Regular' (serifed, proportional), size 10.9091, color #000000



In [26]:
page_font_sizes_counter = Counter(page_font_sizes)
page_font_sizes_counter

Counter({10.909099578857422: 29, 11.9552001953125: 1})

In [27]:
page_font_sizes_counter.most_common(1)[0]

(10.909099578857422, 29)

In [28]:
page_font_sizes_counter.most_common(1)

[(10.909099578857422, 29)]

In [29]:
#it can be modified to just iterate through the blocks instead of the spans

#develop it to pages with low amount of text. the bigest font size could be the title.

def find_titles(page):
    blocks = page.get_text("dict", flags=11)["blocks"]
    page_font_sizes = []
    page_font_names = []
    for b in blocks:  # iterate through the text blocks
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                #print("")
                page_font_sizes.append(s["size"])
                page_font_names.append(s["font"])
                
    page_font_sizes_counter = Counter(page_font_sizes)
    most_common_font_size, most_common_count = page_font_sizes_counter.most_common(1)[0]

    if most_common_count / len(page_font_sizes) > 0.5:
        title_font_sizes = [x for x in page_font_sizes if x > most_common_font_size]
    else:
        return None
    
    title_blocks = []
    for b in blocks:  # iterate through the text blocks
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                if s["size"] in title_font_sizes:
                    title_blocks.append(b)

    return title_blocks

In [30]:
a = find_titles(page)

In [31]:
a

[{'number': 3,
  'type': 0,
  'bbox': (113.38597106933594,
   527.8118286132812,
   406.64715576171875,
   544.9197387695312),
  'lines': [{'spans': [{'size': 11.9552001953125,
      'flags': 20,
      'font': 'LMSans10-Bold',
      'color': 0,
      'ascender': 1.1339999437332153,
      'descender': -0.296999990940094,
      'text': '1.1.1Intracellularcalciumdynamicsanditsmodeling',
      'origin': (113.38597106933594, 541.3690185546875),
      'bbox': (113.38597106933594,
       527.8118286132812,
       406.64715576171875,
       544.9197387695312)}],
    'wmode': 0,
    'dir': (1.0, 0.0),
    'bbox': (113.38597106933594,
     527.8118286132812,
     406.64715576171875,
     544.9197387695312)}]}]

In [32]:
a[0]['bbox']

(113.38597106933594, 527.8118286132812, 406.64715576171875, 544.9197387695312)

In [154]:
title_text_dict = {}
page_blocks = page.get_text("blocks")
page_titles = find_titles(page)



In [155]:
test = fitz.open(pdf_address)
len(test)

67

In [33]:
def is_title(pblock, ptitles):
    if not ptitles:
        return False
    for item in ptitles:
        if item['bbox'] == pblock[:4]:
            #print("bbox of title:", item['bbox'], "\n", "bbox of block:", pblock[:4])
            return True
    return False

In [34]:
def make_master_list(pdf_doc):
    master_list = []

    # Iterate through each page
    for page_num in range(len(pdf_doc)):
        page = pdf_doc.load_page(page_num)

        # Extract text blocks from the page
        text_blocks = page.get_text("blocks")

        page_titles = find_titles(page)

        # Add page number to each block and append to master list
        for block in text_blocks:
            # Create a dictionary for the block
            block_dict = {
                "page": page_num,
                "bbox": block[0:4],  # Position of the text block
                "text": block[4],   # Text content of the block
                "title": is_title(block, page_titles)
            }
            master_list.append(block_dict)

    return master_list

In [37]:
aa = make_master_list(pdf_doc)

In [39]:
aa[2]["page"]

0

In [40]:
doc_list = make_master_list(pdf_doc)

parts_list = []
part_text = []

for index, item in enumerate(doc_list):
    if item['title']:
        title = item["text"]
        start_page = item["page"]
        end_page = doc_list[index+1]["page"]
        part_text = []
    else:
        part_text.append(item["text"])

        part_dict...

In [41]:
part_text

[' \n \n \n \n \nDeclaration  \n \nI hereby certify that I have written the Master Thesis independently, have not used any sources and aids other \nthan those indicated, have clearly identified all contents taken over verbatim or analogously adopted from \nother works, and that the Master Thesis has not been or is not the subject of any other examination \nprocedure, neither in full nor in substantial parts.  \n \n \n \nPlace, Date \n \n \n \n \nSignature \nFreiburg, 04.10.2022\n',
 '<image: DeviceRGB, width: 1000, height: 636, bpc: 8>']

In [207]:
parts_list = []
part_text = []

for i in range(8,len(pdf_doc)):
    page = pdf_doc.load_page(i)
    page_blocks = page.get_text("blocks")
    page_titles = find_titles(page)
    last_title = None

    for b in page_blocks:
        if is_title(b, page_titles):
            title = b[4]
            part_text = []
            start_page = i
        else:
            part_text.append(b[4])
        
        last_title = title
    
    part_dict = {
        'title': title,
        #'pages': f'{start_page+1}-{end_page+1}',  # page number starts from 0 in fitz and 1 in pdf
        'text': part_text,
        #'images': part_images
    }

    parts_list.append(part_dict)
    

page number: 8
page titles: [{'number': 1, 'type': 0, 'bbox': (112.81800079345703, 147.1297607421875, 251.8972930908203, 176.6977996826172), 'lines': [{'spans': [{'size': 20.662500381469727, 'flags': 20, 'font': 'LMSans10-Bold', 'color': 0, 'ascender': 1.1339999437332153, 'descender': -0.296999990940094, 'text': '1Introduction', 'origin': (112.81800079345703, 170.56103515625), 'bbox': (112.81800079345703, 147.1297607421875, 251.8972930908203, 176.6977996826172)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (112.81800079345703, 147.1297607421875, 251.8972930908203, 176.6977996826172)}]}]
page number: 9
page titles: [{'number': 2, 'type': 0, 'bbox': (113.38600158691406, 455.7724914550781, 251.3964385986328, 476.3019104003906), 'lines': [{'spans': [{'size': 14.346199989318848, 'flags': 20, 'font': 'LMSans10-Bold', 'color': 0, 'ascender': 1.1339999437332153, 'descender': -0.296999990940094, 'text': '1.1CalciumImaging', 'origin': (113.38600158691406, 472.04107666015625), 'bbox': (113.3860015869

In [208]:
parts_list

[{'title': '1 Introduction\n',
  'text': ['Recording neural activity is critical for neuroscience research. Neural activity sculpts\nhow neurons transfer information and interact with each other. Different markers or\nindicators can mediate it. Nowadays, a variety of techniques have been developed to\nmeasure brain activity, such as single-channel recording, multi-channel recording, lo-\ncal field potentials (LFPs), electroencephalogram (EEG), magnetoencephalography\n(MEG), functional magnetic resonance imaging (fMRI), positron emission tomog-\nraphy (PET), calcium imaging, optical imaging of intrinsic signals (OIS), volage-\nsensitive dye imaging (VSDI). These techniques can be classified into two main cat-\negories: electrophysiology and imaging techniques. Electrophysiology (patch clamp,\nmulti-channel recording, LFPs, EEG, MEG) measures and manipulates neuronal\ndynamics such as spike trains, EPSC, EPSP, and LFP. Imaging techniques (fMRI,\nPET, calcium imaging, OIS, VSDI) can captu